In [1]:
"""
Description: Perform (backward) selection in currency selection for final Duca basket.
Author: Jeroen van Dijk
Date: 04-11-2020
Maintainer: Jeroen van Dijk
Email: jeroen.vandijk@d-data.nl
Status: Dev
"""
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

from collections import defaultdict
from currency_converter import CurrencyConverter
from datetime import timedelta, date, datetime
from scipy.optimize import minimize

# Set notebook options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 43)
pd.set_option('display.width', 800)
pd.set_option('display.max_colwidth', 15)
pd.set_option('display.precision', 3)

In [2]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def create_original_df(base_currency="EUR"):
    """Create the original dataframe based on ECB data.
   
    Parameters
    ----------
    base_currency : str
        The currency that is used to express other currency exchange rates."""
    c = CurrencyConverter("http://www.ecb.int/stats/eurofxref/eurofxref-hist.zip", 
                          fallback_on_wrong_date=True, 
                          fallback_on_missing_rate=True,
                          decimal=True)

    start_date = date(2000, 1, 1)
    end_date = date.today()

    date_list = []

    for single_date in daterange(start_date, end_date):

        cur_list = []
        cur_list.append(single_date)

        for currency in c.currencies:

            value = c.convert(1, base_currency, currency, date=single_date)
            cur_list.append(float(value))

        date_list.append(cur_list)

    columns_list = ["date"] + list(c.currencies)

    df = pd.DataFrame(date_list, columns=columns_list)
    df = df.sort_values(by="date", ascending=False)
    
    df["date"] = df["date"].map(lambda x: datetime(x.year, x.month, x.day, 0, 0))
    
    return df.sort_values("date").set_index("date")

In [216]:
# Define currencies of interest
p13_currencies = ['USD','EUR','GBP',
                  'CHF','CNY','JPY',
                  'CAD','SEK','NOK',
                  'SGD','AUD','NZD',
                  'ZAR']

# Create dataframe with pre-defined base currency
df = create_original_df("EUR")

In [4]:
obsolete_currencies = ["ISK", "MXN", "ROL", "BRL", "CYP", "EEK", "INR", "ILS", "SIT", "BGN", "LTL", "MTL", "TRL", "LVL", "SKK"]
all_currencies = [c for c in df.columns if c not in obsolete_currencies]

In [5]:
# Determine splits for dev- (train and test) and validation set
train_start = date(2005, 4, 1)
test_start = date(2012, 1, 1)
validation_start = date(2015, 1, 1)
validation_end = date(2020, 10, 1)

# Create dev (train and test) and validation set
train = df.query("date >= @train_start and date < @test_start")
test = df.query("date >= @test_start and date < @validation_start")
dev = pd.concat([train, test])
val = df.query("date >= @validation_start and date < @validation_end")

In [6]:
results = pd.DataFrame()

# Calculate loss function score per currency
for base_currency in all_currencies:
    for currency in all_currencies:
        if currency == base_currency:
            continue
            
        rates = dev[base_currency] / dev[currency]
        normalized_rates = rates / rates.iloc[0]
        
        deviation = np.log(normalized_rates)
        loss_function_score = (deviation**2).mean()

        results = results.append({"currency": currency,
                                  "base_currency": base_currency,
                                  "loss_function_score": loss_function_score},
                                 ignore_index=True)
    
results = results.groupby("currency").median()
results.sort_values("loss_function_score", ascending=True)

,loss_function_score
currency,
CAD,0.015
MYR,0.016
NOK,0.017
HRK,0.020
SEK,0.020
CZK,0.021
EUR,0.021
DKK,0.021
USD,0.021


In [7]:
exchange_table = dict()

for base_currency in all_currencies:
    exchange_table[base_currency] = 1/train[all_currencies].divide(train[base_currency], axis=0)

exchange_table["USD"]

,IDR,HRK,KRW,DKK,SGD,TRY,NOK,THB,GBP,JPY,HUF,CAD,HKD,MYR,USD,CHF,EUR,CNY,NZD,AUD,SEK,RUB,CZK,PLN,RON,PHP,ZAR
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-04-01,1.056e-04,0.175,9.923e-04,0.174,0.604,0.740,0.158,0.026,1.887,0.009,0.005,0.824,0.128,0.263,1.0,0.835,1.296,0.121,0.711,0.771,0.141,0.036,0.043,0.317,0.360,0.018,0.162
2005-04-02,1.055e-04,0.174,9.904e-04,0.174,0.603,0.738,0.158,0.025,1.883,0.009,0.005,0.823,0.128,0.263,1.0,0.833,1.293,0.121,0.709,0.770,0.141,0.036,0.043,0.316,0.359,0.018,0.162
2005-04-03,1.055e-04,0.174,9.886e-04,0.173,0.602,0.736,0.157,0.025,1.879,0.009,0.005,0.822,0.128,0.263,1.0,0.831,1.291,0.121,0.708,0.769,0.141,0.036,0.043,0.315,0.358,0.018,0.161
2005-04-04,1.054e-04,0.174,9.867e-04,0.173,0.600,0.733,0.157,0.025,1.875,0.009,0.005,0.821,0.128,0.263,1.0,0.829,1.288,0.121,0.707,0.768,0.140,0.036,0.043,0.313,0.358,0.018,0.161
2005-04-05,1.053e-04,0.173,9.828e-04,0.172,0.601,0.731,0.157,0.025,1.873,0.009,0.005,0.817,0.128,0.263,1.0,0.824,1.281,0.121,0.704,0.764,0.140,0.036,0.043,0.310,0.356,0.018,0.160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-27,1.086e-04,0.174,8.652e-04,0.176,0.771,0.526,0.168,0.032,1.568,0.013,0.004,0.981,0.129,0.315,1.0,1.070,1.307,0.158,0.774,1.016,0.146,0.032,0.051,0.297,0.305,0.023,0.123
2011-12-28,1.087e-04,0.174,8.651e-04,0.176,0.772,0.526,0.168,0.032,1.567,0.013,0.004,0.985,0.129,0.316,1.0,1.072,1.307,0.158,0.778,1.020,0.146,0.032,0.051,0.297,0.305,0.023,0.123
2011-12-29,1.087e-04,0.171,8.658e-04,0.173,0.767,0.523,0.166,0.031,1.542,0.013,0.004,0.977,0.129,0.315,1.0,1.058,1.289,0.158,0.767,1.007,0.144,0.031,0.050,0.290,0.300,0.023,0.123


In [8]:
reference_date_exhange_rate = train[all_currencies].iloc[0]

def calculate_loss_function_around_one(weights):
    weights = weights / weights.sum()
    amounts = weights * reference_date_exhange_rate

    # TODO: penalize large coefficients?
    # JD: we do not face the problem of large coefficients (yet)
    loss_function_score = 0

    for base_currency in currencies:
        amounts_through_time = amounts * exchange_table[base_currency]
        normalized_amounts_through_time = amounts_through_time.sum(axis=1) / reference_date_exhange_rate[base_currency]
        
        # TODO: other possibilities for loss functions:
        # - Mean Absolute Error MAE
        # - Root Mean Squared Error RMSE
        # - Time-series related loss functions (research)
        
        deviation = np.log(normalized_amounts_through_time)
        loss_function_score += (deviation**2).mean()
    
    return loss_function_score

In [9]:
def calculate_loss_function_vs_t_minus_one(weights):
    weights = weights / weights.sum()
    amounts = weights * reference_date_exhange_rate.loc[p13_currencies]

    loss_function_score = 0

    for base_currency in p13_currencies:
        amounts_through_time = (amounts * exchange_table[base_currency]).sum(axis=1)
        loss_function_score += (np.abs(amounts_through_time - amounts_through_time.shift(1))/amounts_through_time).mean()

    return loss_function_score

starting_point = np.ones(13)/13

result = minimize(calculate_loss_function_vs_t_minus_one, starting_point, options={"disp": True, "maxiter": 100})
duca_mix = pd.Series(result.x/result.x.sum(), index=reference_date_exhange_rate.loc[p13_currencies].index)
print(calculate_loss_function_vs_t_minus_one(duca_mix))
duca_mix.sort_values(ascending=False)

Optimization terminated successfully.
         Current function value: 0.035071
         Iterations: 68
         Function evaluations: 1050
         Gradient evaluations: 75
0.03507095558894311


EUR    0.164
SGD    0.131
GBP    0.102
CNY    0.084
NOK    0.082
SEK    0.075
AUD    0.069
JPY    0.055
CAD    0.053
NZD    0.052
CHF    0.046
USD    0.045
ZAR    0.043
dtype: float64

In [12]:
bounds = [(0,1) for _ in range(len(all_currencies))]

In [18]:
def calculate_loss_function_vs_t_minus_one_with_balancing(weights):
    print("evaluate")
    
    # Ensure we never deviate from 100% total, and only use positive numbers
    weights = np.abs(weights) / np.abs(weights).sum()  
    
    # Calculate the weights after rebalancing for each month
    amounts_by_month = {month: weights / this_day_by_month["EUR"][month].iloc[0][p13_currencies] for month in months}
    
    # Calculate the loss function per base currency, per month
    return sum([sum([(amounts_by_month[month] * diff_by_month[base_currency][month]).sum(axis=1).sum() for month in months]) 
                for base_currency in p13_currencies])

starting_point = np.ones(13)/13
result = minimize(calculate_loss_function_vs_t_minus_one_with_balancing, starting_point, bounds=bounds, options={"disp": True, 
                                                                                                                 "maxiter": 2,
                                                                                                                 })
duca_mix = pd.Series(result.x/result.x.sum(), index=reference_date_exhange_rate.loc[p13_currencies].index)
# print(calculate_loss_function_vs_t_minus_one_with_balancing(duca_mix))
duca_mix.sort_values(ascending=False)

evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate
evaluate


KeyboardInterrupt: 

In [17]:
def calculate_loss_function_vs_t_minus_one_for_period(weights, max_date):
    weights = weights / weights.sum()
    amounts = weights * reference_date_exhange_rate.loc[p13_currencies]

    loss_function_score= 0

    for base_currency in p13_currencies:
        amounts_through_time = (amounts * exchange_table[base_currency].loc[:max_date]).sum(axis=1)
        loss_function_score += (np.abs(amounts_through_time - amounts_through_time.shift(1))/amounts_through_time).mean()

    return loss_function_score

In [94]:
# Everything contains all currencies now
this_day_by_month = {}
previous_day_by_month = {}
diff_by_month = defaultdict(dict)

for base_currency, table in exchange_table.items():
    if base_currency not in all_currencies:
        continue
        
    exchange_table[base_currency]["month"] = exchange_table[base_currency].index.map(lambda x: int(x.strftime("%Y%m")))

    this_day = exchange_table[base_currency]
    previous_day = pd.DataFrame(exchange_table[base_currency].shift(1), 
                                index=exchange_table[base_currency].index)
    previous_day["month"] = previous_day.index.map(lambda x: int(x.strftime("%Y%m")))
    
    this_day_by_month[base_currency] = {month: values[all_currencies] for month, values in this_day.groupby("month")}
    previous_day_by_month[base_currency] = {month: values[all_currencies] for month, values in previous_day.groupby("month")}
    
    for month in this_day_by_month[base_currency].keys():
        diff_by_month[base_currency][month] = np.abs((this_day_by_month[base_currency][month] - previous_day_by_month[base_currency][month]) / 
                                                      previous_day_by_month[base_currency][month])
    
months = list(exchange_table["EUR"]["month"].unique())

In [133]:
def calculate_loss_function_vs_t_minus_one_with_balancing(weights):
    
    # Ensure we never deviate from 100% total, and only use positive numbers
    weights = np.abs(weights) / np.abs(weights).sum()  
    
    # Calculate the weights after rebalancing for each month
    amounts_by_month = {month: weights / this_day_by_month["EUR"][month].iloc[0][all_currencies] for month in months}
    
    # Calculate the loss function per base currency, per month
    return sum([sum([(amounts_by_month[month] * diff_by_month[base_currency][month]).sum(axis=1).sum() for month in months]) 
                for base_currency in all_currencies])

In [207]:
def recalculate_weights(weights):
    """Helper function to recalculate weigths (baseline principle).
   
    Parameters
    ----------
    weights : array
        The initial weights given to all currencies."""
    weights[weights>0] = 1
    weights = weights/weights.sum()
    return weights

In [213]:
def evaluate_exclusion_of_currency(weights, 
                                   currency_set):
    """Perform backward selection of currencies.
   
    Parameters
    ----------
    weights : array
        The initial weights given to all currencies.
    currency_set : set
        The set containing all currencies of interest."""
    # Initialize string giving best currency to exclude
    best_to_exclude = "None"
    # Calculate initial loss function value
    starting_loss_function_value = calculate_loss_function_vs_t_minus_one_with_balancing(weights)
    
    # So far, this is the best solution
    best_loss_function_value = starting_loss_function_value
    print(f"starting loss function value: {best_loss_function_value}")
    print("")

    # Each time we eliminate one currency
    for excluded_currency in currency_set:
        print(f"evaluating {excluded_currency}")

        # We create the new weights,
        new_weights = weights.copy()
        new_weights[new_weights.index==excluded_currency] = 0
        new_weights = recalculate_weights(new_weights)
        
        # and calculate the new loss function value
        new_loss_function_value = calculate_loss_function_vs_t_minus_one_with_balancing(new_weights)
        print(f"new loss function value: {new_loss_function_value}")

        # Check whether the new loss function value is lower than the (so far) best one, and
        # the improvement needs to be > .5%
        if new_loss_function_value <= best_loss_function_value and new_loss_function_value < .995*starting_loss_function_value:
            print(f"better solution found by excluding {excluded_currency}")
            print(f"loss function decreased by {(best_loss_function_value-new_loss_function_value)/best_loss_function_value*100:.2f}% compared to current best solution")
            # If so, save these results
            best_loss_function_value = new_loss_function_value
            best_to_exclude = excluded_currency
        print("")
        
    if best_to_exclude == "None":
        print("No currency needed to be removed.")
    else:
        print(f"{best_to_exclude} is removed from currency set")
    print("")
    print(f"--------- new iteration ---------")
    print("")
    
    # Exclude the currency
    currency_set.remove(best_to_exclude)
    return starting_loss_function_value, best_loss_function_value, best_to_exclude, currency_set

In [215]:
weights = pd.Series(np.ones(len(all_currencies))/len(all_currencies), index=reference_date_exhange_rate.loc[all_currencies].index)
currency_set = set(all_currencies)
max_excluded_currencies = 27

for i in range(0, max_excluded_currencies):
    _, _, best_to_exclude, currencies = evaluate_exclusion_of_currency(weights, currency_set)
    if best_to_exclude == "None":
        continue
    weights[weights.index==best_to_exclude] = 0
    weights = recalculate_weights(weights)

starting loss function value: 143338.79797809376

evaluating IDR
new loss function value: 23172.962980856497
better solution found by excluding IDR
loss function decreased by 83.83% compared to current best solution

evaluating HRK
new loss function value: 148794.93714275828

evaluating KRW
new loss function value: 132699.4163045596

evaluating DKK
new loss function value: 148796.06917962988

evaluating SGD
new loss function value: 148837.15387720184

evaluating TRY
new loss function value: 148829.9144931999

evaluating NOK
new loss function value: 148774.93370797014

evaluating THB
new loss function value: 148423.37783083392

evaluating GBP
new loss function value: 148844.59799098692

evaluating JPY
new loss function value: 147215.5684671395

evaluating HUF
new loss function value: 145844.7285456512

evaluating CAD
new loss function value: 148837.22579296524

evaluating HKD
new loss function value: 148760.2570556347

evaluating MYR
new loss function value: 148814.12712404705

evaluati

In [309]:
# TODO: Different steps for creating new weights
new_weights[new_weights>0] = (1-possible_weights_included_currency[0])/(no_currencies-1)
new_weights[included_currency] = possible_weights_included_currency[0]
new_weights

IDR    0.00
HRK    0.00
KRW    0.00
DKK    0.00
SGD    0.00
TRY    0.00
NOK    0.00
THB    0.00
GBP    0.95
JPY    0.00
HUF    0.00
CAD    0.00
HKD    0.00
MYR    0.00
USD    0.00
CHF    0.00
EUR    0.05
CNY    0.00
NZD    0.00
AUD    0.00
SEK    0.00
RUB    0.00
CZK    0.00
PLN    0.00
RON    0.00
PHP    0.00
ZAR    0.00
dtype: float64

In [300]:
no_currencies = (new_weights>0).sum()
possible_weights_included_currency = np.arange(0, 1/no_currencies, 1/no_currencies/10) + 1/no_currencies/10
possible_weights_included_currency
for possible_weight in possible_weights_included_currency:
    new_weights[new_weights>0] = (1-possible_weight)/(no_currencies-1)
    new_weights[included_currency] = possible_weight

0.05
others have weight: 0.95

0.1
others have weight: 0.9

0.15000000000000002
others have weight: 0.85

0.2
others have weight: 0.8

0.25
others have weight: 0.75

0.3
others have weight: 0.7

0.35000000000000003
others have weight: 0.6499999999999999

0.4
others have weight: 0.6

0.45
others have weight: 0.55

0.5
others have weight: 0.5



In [332]:
def evaluate_inclusion_of_currency(weights, 
                                   currency_set):
    """Perform forward selection of currencies.
   
    Parameters
    ----------
    weights : array
        The initial weights given to all currencies.
    currency_set : set
        The set containing all currencies of interest."""

    # Initialize string giving best currency to exclude
    best_to_include = "None"
    # Calculate initial loss function value
    starting_loss_function_value = calculate_loss_function_vs_t_minus_one_with_balancing(weights)

    # So far, this is the best solution
    best_loss_function_value = starting_loss_function_value
    print(f"starting loss function value: {best_loss_function_value}")

    # Each time we include one currency
    for included_currency in currency_set:
        print(f"\nincluding {included_currency}")

        # We create the new weights,
        new_weights = weights.copy()
        new_weights[new_weights.index==included_currency] = 1
        new_weights = recalculate_weights(new_weights)

        # and calculate the new loss function value
        new_loss_function_value = calculate_loss_function_vs_t_minus_one_with_balancing(new_weights)
        print(f"new loss function value: {new_loss_function_value}")
        
        # TODO: Check boundaries
        # Check whether the new loss function value is lower than the (so far) best one, and
        # the improvement needs to be > .5%
        if new_loss_function_value <= 1.05*best_loss_function_value and new_loss_function_value < 1.05*starting_loss_function_value:
            print(f"\nbetter solution found by including {included_currency}")
            print(f"loss function decreased by {(best_loss_function_value-new_loss_function_value)/best_loss_function_value*100:.2f}% compared to current best solution\n")
            # If so, save these results
            best_loss_function_value = new_loss_function_value
            best_to_include = included_currency
        print("")

    if best_to_include == "None":
        print("No currency needed to be included.")
    else:
        print(f"{best_to_include} is added to currency set\n")
        # Exclude the currency
        currency_set.remove(best_to_include)
    return starting_loss_function_value, best_loss_function_value, best_to_include, currency_set

In [314]:
# Create weights series and specificy first currency to include
weights = pd.Series(0, index=reference_date_exhange_rate.loc[all_currencies].index)
first_currency = "GBP"
weights[first_currency] = 1
currency_set = set(all_currencies)
currency_set.remove(first_currency)

# Set maximum number of currencies to include
max_currencies_to_include = 14

for i in range(0, max_currencies_to_include):
    _, _, best_to_include, currencies = evaluate_inclusion_of_currency(weights, currency_set)
    if best_to_include == "None":
        break
    weights[weights.index==best_to_include] = 0
    weights = recalculate_weights(weights)

starting loss function value: 187.99764287142642

including IDR
new loss function value: 1633919.2527745673

including HRK
new loss function value: 833.5886698438062

including KRW
new loss function value: 210075.35956642678

including DKK
new loss function value: 818.8721905129803

including SGD
new loss function value: 284.7711220777229

including TRY
new loss function value: 378.88311410315947

including NOK
new loss function value: 1093.633322089875

including THB
new loss function value: 5663.859724860598

including JPY
new loss function value: 21365.38145288797

including HUF
new loss function value: 39186.30043223617

including CAD
new loss function value: 283.8362171529786

including HKD
new loss function value: 1284.4298024501534

including MYR
new loss function value: 584.1189130893704

including USD
new loss function value: 248.7347689771737

including CHF
new loss function value: 275.53478340838944

including EUR
new loss function value: 191.2882387482108

better solution f

In [330]:
def evaluate_inclusion_of_currency_difference_weights(weights, 
                                                      currency_set):
    """Perform forward selection of currencies.
   
    Parameters
    ----------
    weights : array
        The initial weights given to all currencies.
    currency_set : set
        The set containing all currencies of interest."""

    # Initialize string giving best currency to exclude
    best_to_include = "None"
    # Initialize string giving best currency weights
    best_weights = weights.copy()
    # Calculate initial loss function value
    starting_loss_function_value = calculate_loss_function_vs_t_minus_one_with_balancing(weights)

    # So far, this is the best solution
    best_loss_function_value = starting_loss_function_value
    print(f"starting loss function value: {best_loss_function_value}")

    # Each time we include one currency
    for included_currency in currency_set:
        print(f"\nincluding {included_currency}\n")
        
        new_weights = weights.copy()
        no_currencies = (new_weights>0).sum()+1
        possible_weights_included_currency = np.arange(0, 1/no_currencies, 1/no_currencies/10) + 1/no_currencies/10
        
        for possible_weight in possible_weights_included_currency:
            new_weights[new_weights>0] = (1-possible_weight)/(no_currencies-1)
            new_weights[included_currency] = possible_weight
        
            # We create the new weights,
#             new_weights = weights.copy()
#             new_weights[new_weights.index==included_currency] = 1
#             new_weights = recalculate_weights(new_weights)

            # and calculate the new loss function value
            new_loss_function_value = calculate_loss_function_vs_t_minus_one_with_balancing(new_weights)
            print(f"weight {included_currency}: {possible_weight}")
            print(f"other weight(s): {(1-possible_weight)/(no_currencies-1)}")
            print(f"new loss function value: {new_loss_function_value}\n")

            # TODO: Check boundaries
            # Check whether the new loss function value is lower than the (so far) best one, and
            # the improvement needs to be > .5%
            if new_loss_function_value <= 1.05*best_loss_function_value and new_loss_function_value < 1.05*starting_loss_function_value:
                print(f"\nbetter solution found by including {included_currency}")
                print(f"loss function decreased by {(best_loss_function_value-new_loss_function_value)/best_loss_function_value*100:.2f}% compared to current best solution\n")
                print(best_weights)
                # If so, save these results
                best_weights = new_weights
                best_loss_function_value = new_loss_function_value
                best_to_include = included_currency

    if best_to_include == "None":
        print("No currency needed to be included.")
    else:
        print(f"{best_to_include} is added to currency set\n")
        # Exclude the currency
        currency_set.remove(best_to_include)
    return starting_loss_function_value, best_loss_function_value, best_to_include, best_weights, currency_set

In [331]:
# Create weights series and specificy first currency to include
weights = pd.Series(0, index=reference_date_exhange_rate.loc[all_currencies].index)
first_currency = "GBP"
weights[first_currency] = 1
currency_set = set(all_currencies)
currency_set.remove(first_currency)

# Set maximum number of currencies to include
max_currencies_to_include = 14

for i in range(0, max_currencies_to_include):
    _, _, best_to_include, currencies = evaluate_inclusion_of_currency_difference_weights(weights, currency_set)
    if best_to_include == "None":
        break
    weights[weights.index==best_to_include] = 0
    weights = recalculate_weights(weights)

starting loss function value: 187.99764287142642

including IDR

weight IDR: 0.05
other weight(s): 0.95
new loss function value: 163561.12315604097

weight IDR: 0.1
other weight(s): 0.9
new loss function value: 326934.24866921053



KeyboardInterrupt: 

In [30]:
def get_libra_mix(reference_date_exhange_rate):
    result = pd.Series(np.zeros(len(reference_date_exhange_rate)), index=reference_date_exhange_rate.index)
    
    result["USD"] = .5
    result["EUR"] = .18
    result["JPY"] = .14
    result["GBP"] = .11
    result["SGD"] = .07
    
    return result

def get_sdr_mix(reference_date_exhange_rate):
    result = pd.Series(np.zeros(len(reference_date_exhange_rate)), index=reference_date_exhange_rate.index)
    
    result["USD"] = .4173
    result["EUR"] = .3093
    result["CNY"] = .1092
    result["JPY"] = .0833
    result["GBP"] = .0809
    
    return result

assert get_libra_mix(reference_date_exhange_rate).sum() == 1
assert get_sdr_mix(reference_date_exhange_rate).sum() == 1